In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
file_path = 'C:/Users/User/OIBSIP/sentiment_analysis/data/Twitter_Data.csv/Twitter_Data.csv'
data = pd.read_csv(file_path)

# Preprocessing function
def preprocess_text_safe(text):
    if isinstance(text, str):
        # Lowercase the text
        text = text.lower()
        # Remove URLs
        text = re.sub(r'http\S+', '', text)
        # Remove punctuation and numbers
        text = re.sub(r'[^a-z\s]', '', text)
        # Tokenize and remove stop words manually
        stop_words = set(['the', 'is', 'in', 'and', 'to', 'has', 'that', 'with', 'for', 'it', 'as', 'on', 'at', 'this', 'by', 'an', 'be', 'or', 'are', 'was', 'from'])
        tokens = text.split()
        tokens = [word for word in tokens if word not in stop_words]
        return ' '.join(tokens)
    else:
        return ""

# Apply preprocessing
data['processed_text'] = data['clean_text'].apply(preprocess_text_safe)

# Drop rows with missing values in 'category'
data = data.dropna(subset=['category'])

# Define features and labels
X = data['processed_text']
y = data['category']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Initialize and train the model
model = LinearSVC()
model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the evaluation results
evaluation_results = {
    'Accuracy': accuracy,
    'Classification Report': report,
    'Confusion Matrix': conf_matrix
}

evaluation_results


{'Accuracy': 0.9422610829881883,
 'Classification Report': '              precision    recall  f1-score   support\n\n        -1.0       0.92      0.87      0.90      7230\n         0.0       0.95      0.98      0.96     10961\n         1.0       0.95      0.95      0.95     14404\n\n    accuracy                           0.94     32595\n   macro avg       0.94      0.93      0.94     32595\nweighted avg       0.94      0.94      0.94     32595\n',
 'Confusion Matrix': array([[ 6317,   272,   641],
        [   95, 10737,   129],
        [  451,   294, 13659]], dtype=int64)}